Rebuild street graph
====================
Loads a street graph and reallocates roads space, e.g., into a network of one-way streets

In [30]:
import snman

PERIMETER = 'aarau_extended'
# Set SAVE_TO_DEBUG = True for saving the results into the _debug folder
# which is automatically used in the QGIS files
SAVE_TO_DEBUG = True

# Set these paths according to your own setup
data_directory = '/Users/svenruf/OneDrive - ETH Zurich/masterarbeit/Code/data_directory_aarau/'
inputs_path = data_directory + 'inputs/'
process_path = data_directory + 'process/' + PERIMETER + '/'

if SAVE_TO_DEBUG:
    export_path = data_directory + 'outputs/' + '_debug' + '/'
else:
    export_path = data_directory + 'outputs/' + PERIMETER + '/'

In [ ]:
# =====================================================================================
# LOAD DATA
# =====================================================================================

print('Load street graph')
G = snman.io.load_street_graph(process_path + 'edges_all_attributes.gpkg', process_path + 'nodes_all_attributes.gpkg')

print('Load rebuilding regions')
# Polygons that define which streets will be reorganized
rebuilding_regions_gdf = snman.io.load_rebuilding_regions(
    inputs_path + 'rebuilding_regions/rebuilding_regions.gpkg'
)

In [ ]:
# =====================================================================================
# GIVEN LANES
# =====================================================================================

if 1:
    print('Set given lanes')
    snman.set_given_lanes(G, bidirectional_for_dead_ends=False)

if 1:
    print('Create directed graph of given lanes')
    G_minimal_graph_input = snman.create_given_lanes_graph(G, hierarchies_to_remove={snman.hierarchy.HIGHWAY})

In [ ]:
# =====================================================================================
# REBUILD
# =====================================================================================

if 1:
    print('Rebuild regions')
    snman.owtop.rebuild_regions(G, rebuilding_regions_gdf, verbose=True)
    snman.generate_lane_stats(G, lanes_attribute=snman.constants.KEY_LANES_DESCRIPTION_AFTER)

In [ ]:
# =====================================================================================
# EXPORT
# =====================================================================================

if 1:
    print('Export network without lanes')
    snman.export_street_graph(G, export_path + 'edges_all_attributes.gpkg', export_path + 'nodes_all_attributes.gpkg')
    snman.export_street_graph(G, export_path + 'edges.gpkg', export_path + 'nodes.gpkg',
        edge_columns=snman.constants.EXPORT_EDGE_COLUMNS
    )

if 0:
    print('Export OSM XML')
    snman.export_osm_xml(G, export_path + 'new_network.osm',{
        'highway', 'lanes', 'lanes:forward', 'lanes:backward', 'lanes:both_ways',
        'cycleway', 'cycleway:lane', 'cycleway:left', 'cycleway:left:lane', 'cycleway:right', 'cycleway:right:lane',
        'bus:lanes:backward', 'bus:lanes:forward', 'vehicle:lanes:backward', 'vehicle:lanes:forward',
        'maxspeed', 'oneway',
        '_connected_component'
    }, uv_tags=True, tag_all_nodes=True)

if 1:
    print('Export network with lanes')
    snman.export_street_graph_with_lanes(G, 'ln_desc', export_path + 'edges_lanes.shp', scaling=2)
    snman.export_street_graph_with_lanes(G, 'ln_desc_after', export_path + 'edges_lanes_after.shp', scaling=2)